In [ ]:
import random
import itertools
import copy
import numpy as np
from dice import Dice

In [ ]:
pdb

In [ ]:
d4 = Dice(4)
d6 = Dice(6)
d8 = Dice(8)
d10 = Dice(10)
d12 = Dice(12)
d20 = Dice(20)
d100 = Dice(100)

In [ ]:
class Damage:
    def __init__(self, rolls, flat_bonus):
        self.rolls = rolls
        self.flat_bonus = flat_bonus
    def roll(self):
        rolls = sum([random.randint(1, roll) for roll in self.rolls])
        return rolls + self.flat_bonus

In [ ]:
class Attack:
    def __init__(self, to_hit, damage):
        self.to_hit = to_hit
        self.damage = damage
    def apply_damage(self, to_hit, damage, enemy):
        if to_hit >= enemy.ac:
            enemy.hp -= damage
        
    def roll_hit(self, advantage = False, disadvantage = False, enemy=None):
        if advantage and disadvantage:
            advantage = disadvantage = False
        roll1 = self.to_hit + d20.roll()
        if advantage:
            roll2 = self.to_hit + d20.roll()
            self.apply_damage(max(roll1, roll2), self.damage(), enemy)
        if disadvantage:
            roll2 = self.to_hit + d20.roll()
            self.apply_damage(min(roll1, roll2), self.damage(), enemy)
        self.apply_damage(roll1, self.damage.roll(), enemy)

In [ ]:
class Character:
    def __init__(self, hp, ac, attacks, stats, name, spells = [], spell_slots = [], spellcasting_modifier = 0):
        self.hp = hp
        self.max_hp = hp
        self.ac = ac
        self.spells = spells
        self.spell_slots = spell_slots
        self.max_spell_slots = copy.copy(spell_slots)
        if not isinstance(attacks, list):
            attacks = [attacks]
        self.attacks = attacks
        self.stats = stats
        self.name = name
        self.spellcasting_modifier = spellcasting_modifier
    def roll_init(self):
        return self.stats['dex'] + d20.roll() + self.stats['dex']/100.
    def take_hit(self, to_hit, damage):
        if to_hit >= self.ac:
            self.hp -= damage

In [ ]:
### ability_by_method comes from players guides on rpgbot.net
### need to finish entering in class proficiencies  for the majority of the classes, otherwise all info from phb
### classes table entered

classes = {
'barbarian':{
    'hit_die':'d12',
    'primary_ability': 'str',
    'saving_throw_prof':['str', 'con'],
    'arm_&_wep_prof':['light armor', 'medium armor', 'shields', 'simple weapons', 'martial weapons'],
    'ability_by_method':{
        'pb':{
            'str': 15, 
            'dex': 14,
            'con': 14,
            'int':  8, 
            'wis': 10,
            'cha': 10}, 
        'sa':{
            'str': 15, 
            'dex': 13, 
            'con': 14, 
            'int':  8, 
            'wis': 10, 
            'cha':12}}}, 
'bard':{
    'hit_die':'d8',
    'primary_ability': 'cha',
    'saving_throw_prof':['dex', 'cha'],
    'arm_&_wep_prof':['light armor', 'simple weapons', 'hand crossbows', 'long swords', 'rapiers', 'short swords'],
    'ability_x_method':{
        'pb':{
            'str':  8, 
            'dex': 15,
            'con': 13,
            'int': 12 , 
            'wis': 10,
            'cha': 15}, 
        'sa':{
            'str':  8, 
            'dex': 14, 
            'con': 13, 
            'int': 12, 
            'wis': 10, 
            'cha': 15}}},
'cleric':{
    'hit_die':'d8',
    'primary_ability': 'wis',
    'saving_throw_prof':['wis', 'cha'],
    #'arm_&_wep_prof':['light armor', 'medium armor', 'shields', 'simple weapons', 'martial weapons'],
    'ability_x_method':{
        'pb':{
            'light armor':{
                'str':  8, 
                'dex': 14,
                'con': 14,
                'int':  8, 
                'wis': 15,
                'cha': 12},
            'medium armor':{
                'str':  8, 
                'dex': 14,
                'con': 14,
                'int':  8, 
                'wis': 15,
                'cha': 12},
            'heavy armor':{
                'str': 14, 
                'dex':  8,
                'con': 14,
                'int':  8, 
                'wis': 15,
                'cha': 12}},                     
        'sa':{
            'light armor':{
                'str':  8, 
                'dex': 14,
                'con': 13,
                'int': 12, 
                'wis': 15,
                'cha': 10},
            'medium armor':{
                'str': 12, 
                'dex': 13,
                'con': 14,
                'int': 10, 
                'wis': 15,
                'cha':  8},
            'heavy armor':{
                'str': 13, 
                'dex':  8,
                'con': 14,
                'int': 12, 
                'wis': 15,
                'cha': 10}}}},                     
'druid':{
    'hit_die':'d8',
    'primary_ability': 'wis',
    'saving_throw_prof':['int', 'wis'],
    #'arm_&_wep_prof':['light armor', 'medium armor', 'shields', 'simple weapons', 'martial weapons'],
    'ability_x_method':{
        'pb':{
            'str':  8, 
            'dex': 14,
            'con': 14,
            'int': 12 , 
            'wis': 15,
            'cha':  8}, 
        'sa':{
            'str': 10, 
            'dex': 13, 
            'con': 14, 
            'int': 12, 
            'wis': 15, 
            'cha':  8}}},
'fighter':{
    'hit_die':'d10',
    'primary_ability': ['str','dex'],
    'saving_throw_prof':['str', 'con'],
    #'arm_&_wep_prof':['light armor', 'medium armor', 'shields', 'simple weapons', 'martial weapons'],
    'ability_x_method':{
        'pb':{
            'Strength_based':{
                'str': 15,
                'dex': 10,
                'con': 15,
                'int': 10,
                'wis': 12,
                'cha':  8},
            'E_Knight/Psi_War_based':{
                'str': 15,
                'dex':  8,
                'con': 14,
                'int': 14,
                'wis': 12,
                'cha':  8},
            'Finesse/Archery_based':{
                'str':  8,
                'dex': 15,
                'con': 15,
                'int': 10,
                'wis': 13,
                'cha': 10}},                     
        'sa':{
            'str_based':{
                'str': 15,
                'dex':  8,
                'con': 14,
                'int': 10,
                'wis': 13,
                'cha': 12},
            'E_Knight/Psi_War_based':{
                'str': 15,
                'dex':  8,
                'con': 14,
                'int': 13,
                'wis': 12,
                'cha': 10},
            'Finesse/Archery_based':{
                'str':  8,
                'dex': 15,
                'con': 14,
                'int': 10,
                'wis': 13,
                'cha': 12}}}},                                   
'monk':{
    'hit_die':'d8',
    'primary_ability': ('dex', 'wis'),
    'saving_throw_prof':['str', 'dex'],
    #'arm_&_wep_prof':['light armor', 'medium armor', 'shields', 'simple weapons', 'martial weapons'],
    'ability_x_method':{
        'pb':{
            'str':  8,
            'dex': 15,
            'don': 14,
            'dnt': 10,
            'wis': 15,
            'cha':  8}, 
        'sa':{
            'str': 12,
            'dex': 15,
            'con': 14,
            'int': 10,
            'wis': 13,
            'cha':  8}}},
'paladin':{
    'hit_die':'d10',
    'primary_ability': ('str', 'cha'),
    'saving_throw_prof':['wis', 'cha'],
    #'arm_&_wep_prof':['light armor', 'medium armor', 'shields', 'simple weapons', 'martial weapons'],
    'ability_x_method':{
        'pb':{
            'str_based':{
                'str': 15,
                'dex':  8,
                'con': 15,
                'int':  8,
                'wis':  8,
                'cha': 15},
            'dex_based':{
                'str':  8,
                'dex': 15,
                'con': 15,
                'int':  8,
                'wis':  8,
                'cha': 15},
            'cha_based':{
                'str': 15,
                'dex':  8,
                'don': 15,
                'dnt':  8,
                'wis':  8,
                'cha': 15}},                     
        'sa':{
            'str_based':{
                'str': 15,
                'dex':  8,
                'don': 13,
                'int': 10,
                'wis': 12,
                'cha': 14},
            'dex_based':{
                'str':  8,
                'dex': 15,
                'con': 13,
                'int': 10,
                'wis': 12,
                'cha': 14},
            'cha_based':{
                'str': 14,
                'dex':  8,
                'con': 13,
                'int': 10,
                'wis': 12,
                'cha': 15}}}},                                          
'ranger':{
    'hit_die':'d10',
    'primary_ability': ('dex', 'wisd'),
    'saving_throw_prof':['str', 'dex'],
    #'arm_&_wep_prof':['light armor', 'medium armor', 'shields', 'simple weapons', 'martial weapons'],
    'ability_x_method':{
        'pb':{
            'dex_based':{
                'str':  8,
                'dex': 15,
                'con': 14,
                'int': 10,
                'wis': 15,
                'cha':  8},
            'str_based':{
                'str': 15,
                'dex': 14,
                'con': 14,
                'int':  8,
                'wis': 12,
                'cha':  8},
            'wis_based':{
                'str':  8,
                'dex': 14,
                'con': 14,
                'int':  8,
                'wis': 15,
                'cha': 12}},                     
        'sa':{
            'dex_based':{
                'str': 15,
                'dex':  8,
                'con': 13,
                'int': 10,
                'wis': 12,
                'cha': 14},
            'str_based':{
                'str':  8,
                'dex': 15,
                'con': 13,
                'int': 10,
                'wis': 12,
                'cha': 14},
            'wis_based':{
                'str': 14,
                'dex':  8,
                'con': 13,
                'int': 10,
                'wis': 12,
                'cha': 15}}}},                                  
'rogue':{
    'hit_die':'d8',
    'primary_ability': 'dex',
    'saving_throw_prof':['dex', 'int'],
    #'arm_&_wep_prof':['light armor', 'medium armor', 'shields', 'simple weapons', 'martial weapons'],
    'ability_x_method':{
        'pb':{
            'main':{
                'str':  8,
                'dex': 15,
                'con': 14,
                'int': 11,
                'wis': 12,
                'cha': 12},
            'arcane_trickster':{
                'str': 10,
                'dex': 15,
                'con': 14,
                'int': 14,
                'wis': 10,
                'cha': 10},                    
        'sa':{
            'main':{
                'str':  8,
                'dex': 15,
                'con': 14,
                'int': 10,
                'wis': 13,
                'cha': 12},
            'arcane_trickster':{
                'str':  8,
                'dex': 15,
                'con': 13,
                'int': 14,
                'wis': 12,
                'cha': 10}}}}},                                  
'sorcerer':{
    'hit_die':'d6',
    'primary_ability': 'cha',
    'saving_throw_prof':['con', 'cha'],
    #'arm_&_wep_prof':['light armor', 'medium armor', 'shields', 'simple weapons', 'martial weapons'],
    'ability_x_method':{
        'pb':{
             'str':  8,
             'dex': 14,
             'con': 14,
             'int': 10,
             'wis': 10,
             'cha': 15}, 
        'sa':{
             'str':  8,
             'dex': 13,
             'con': 14,
             'int': 12,
             'wis': 10,
             'cha': 15}}},
'warlock':{
    'hit_die':'d8',
    'primary_ability': 'cha',
    'saving_throw_prof':['wis', 'cha'],
    #'arm_&_wep_prof':['light armor', 'medium armor', 'shields', 'simple weapons', 'martial weapons'],
    'ability_x_method':{
        'pb':{
            'chain/talisman/tome':{
                'str':  8,
                'dex': 14,
                'con': 14,
                'int': 10,
                'wis': 10,
                'cha': 15},
            'blade':{
                'str': 10,
                'dex': 15,
                'con': 14,
                'int':  8,
                'wis':  8,
                'cha': 14},                    
        'sa':{
            'chain/talisman/tome':{
                'str':  8,
                'dex': 13,
                'con': 14,
                'int': 12,
                'wis': 10,
                'cha': 15},
            'blade':{
                'str': 10,
                'dex': 14,
                'con': 13,
                'int': 12,
                'wis':  8,
                'cha': 15}}}}},                        
'wizard':{
    'hit_die':'d6',
    'primary_ability': 'int',
    'saving_throw_prof':['int', 'wis'],
    #'arm_&_wep_prof':['light armor', 'medium armor', 'shields', 'simple weapons', 'martial weapons'],
    'ability_x_method':{
        'pb':{
            'str':  8,
            'dex': 14,
            'con': 14,
            'int': 15,
            'wis': 12,
            'cha':  8}, 
        'sa':{
            'str':  8,
            'dex': 13,
            'con': 14,
            'int': 15,
            'wis': 12,
            'cha': 10}}}}

In [ ]:
class HealingWord:
    def __init__(self):
        self.time = 'bonus'
        self.type = 'heal'
        pass
    def cast(self, targets, ava_spell_slots, spellcasting_modifier, *args, **kwargs):
        spell_level = 1 + max([i for i,v in enumerate(ava_spell_slots) if v>0])
        amount_healed = sum([d4.roll() for _ in range(spell_level)]) + spellcasting_modifier
        targets.hp += amount_healed
        return spell_level

In [ ]:
class MagicMissile:
    def __init__(self):
        self.time = 'action'
        self.type = 'attack'
        pass
    def cast(self, enemies, ava_spell_slots, *args, **kwargs):
        spell_level = 1 + max([i for i,v in enumerate(ava_spell_slots) if v>0])
        missle = 2 + spell_level
        if not isinstance(enemies, list):
            enemies = [enemies] * missle
        for enemy in enemies:
            damage = d4.roll() + 1
            enemy.hp -= damage
        return spell_level

In [ ]:
fighter = Character(
    hp = 13, ac = 18, attacks = Attack(to_hit = 4, damage = Damage(rolls = [6], flat_bonus = 2)), stats = {'dex':-1},
    name = 'fighter')
cleric = Character(
    hp = 10, ac = 18, attacks = Attack(to_hit = 5, damage = Damage(rolls = [8], flat_bonus = 5)), 
    spells = [set([HealingWord()]), ], spell_slots = [2,], stats = {'dex':-1},
    spellcasting_modifier = 5, name = 'cleric')
warlock = Character(
    hp = 11, ac = 15, attacks = Attack(to_hit = 5, damage = Damage(rolls = [6], flat_bonus = 3)), 
    stats = {'dex':2},
    name = 'warlock')
sorcerer = Character(
    hp = 9, ac = 15, attacks = Attack(to_hit = 5, damage = Damage(rolls = [10], flat_bonus = 0)),
    spells = [set([MagicMissile()]), ], spell_slots = [1,]
    , stats = {'dex':2},
    name = 'sorcerer')
rogue = Character(
    hp = 11, ac = 14, attacks = [
        Attack(to_hit = 5, damage = Damage(rolls = [6], flat_bonus = 3)),
        Attack(to_hit = 5, damage = Damage(rolls = [6], flat_bonus = 0)),
    ], stats = {'dex':3}, name = 'rogue')
team_PCs = [fighter, cleric, rogue, warlock]
# team_NPCs = [fighter2, cleric2, warlock2, rogue2]

In [ ]:
sorcerer.max_spell_slots

In [ ]:
def roll_init(x):
    init = [xi.roll_init() for xi in x]
    order = np.argsort(init)
    init_order = [x[i] for i in order[::-1]]
    return init_order
    

In [ ]:
def check_death(state):
    for char_group, return_val in [['npcs', 0], ['pcs', 1]]:
        alive = [c for c in state[char_group] if c.hp > 0]
        if len(alive) == 0:
            return return_val
    return -1

In [ ]:
def check_heal(time, state, c):
    available_spells = possible_spells(c, time, 'heal')
    ally = get_target(c, state, 'ally', 'dead')
    if len(available_spells) == 0 or ally == -1:
        return -1
    spell = available_spells[0]
    slot = spell.cast(ally, c.spell_slots, c.spellcasting_modifier)
    return slot

In [ ]:
def possible_spells(c, time, spell_type):
    available_spells = [spells for spells, slots in zip(c.spells, c.spell_slots) if slots > 0]

    available_spells = list(itertools.chain(*available_spells))
    available_spells = [spell for spell in available_spells if spell.time == time and spell.type == spell_type]
    
    return available_spells

In [ ]:
def get_available_targets(characters, target_hps):
    if target_hps == 'dead':
        return [c for c in characters if c.hp == 0]
    if target_hps == 'alive':
        return [c for c in characters if c.hp >= 1]
    return characters

In [ ]:
def get_target(character, state, target_type = 'enemy', target_hps = None):
    if target_type == 'enemy':
        target_hps = 'alive'
    if bool(character in state['pcs']) ^ bool(target_type == 'ally'):
        targets = get_available_targets(state['npcs'], target_hps)
    else:
        targets = get_available_targets(state['pcs'], target_hps)
    if len(targets) > 0:
        return targets[0]
    else:
        return -1


In [ ]:
def combat(pcs, npcs, strategy = None):
    all_characters = pcs + npcs
    state = {'pcs': pcs, 'npcs': npcs}
    initiative = roll_init(all_characters)
    [setattr(c, 'hp', c.max_hp) for c in initiative]
    [setattr(c, 'spell_slots', c.max_spell_slots) for c in initiative]
    last_attacked = -1
    for _ in range(100):
        for character in initiative:
            if (character.hp <= 0):
                continue
            c = character
            if (strategy == 'last'):
                if (last_attacked == -1 or last_attacked.hp < 0):
                    enemy = get_target(c, state, 'enemy', last_attacked)
                else:
                    enemy = last_attacked
            else:
                enemy = get_target(c, state, 'enemy')
            spells_available = possible_spells(c, 'action', 'attack')
            if len(spells_available) > 0:
                to_cast = spells_available[0]
                slot_used = to_cast.cast(enemy, c.spell_slots, c.spellcasting_modifier)
                c.spell_slots[slot_used-1] = c.spell_slots[slot_used-1]-1
            else:
                for attack in character.attacks:
                    attack.roll_hit(enemy = enemy)
                    end_val = check_death(state)
                    if end_val >= 0:
                        return end_val
            end_val = check_death(state)
            if end_val >= 0:
                return end_val
            slot_used = check_heal('bonus', state, c)
            if (slot_used != -1):
                c.spell_slots[slot_used-1] = c.spell_slots[slot_used-1]-1
    return -1

In [ ]:
# sum([combat(team_PCs, team_NPCs) for _ in range(100)])/100

In [ ]:
team_PCs = [fighter, cleric, rogue, warlock]
# team_NPCs = [fighter2, cleric2, warlock2, rogue2]

In [ ]:
cleric_team = [copy.deepcopy(cleric) for _ in range(4)]
rogue_team = [copy.deepcopy(rogue) for _ in range(4)]
sorc_team = [copy.deepcopy(sorcerer) for _ in range(4)]
warlock_team = [copy.deepcopy(warlock) for _ in range(4)]
fighter_team = [copy.deepcopy(fighter) for _ in range(4)]
# "0.97" means team 1 wins 97% of the team against team 2
print(sum([combat(sorc_team, cleric_team, 'last') for _ in range(100)])/100)
# combat(cleric_team, sorc_team)
sum([combat(sorc_team, rogue_team) for _ in range(100)])/100

In [ ]:
sorc_team[0].max_spell_slots